### Vehicle Tracking and Counting Using YOLOv9

The YOLOv9 series introduces four models:

1. YOLOv9-s (small)
2. YOLOv9-m (medium)
3. YOLOv9-c (compact)
4. YOLOv9-e (extended)

In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install pandas
!pip install torch torchvision
!pip install matplotlib

### Download Video from Goolge Drive

In [ ]:
import os
import requests

# https://drive.google.com/file/d/1RLmqoMl0WTtng_vhvWt-ATHrGqzjtgr6/view?usp=share_link

def download_file_from_google_drive(file_id, dest_path):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, dest_path)

    # Check if the file is downloaded successfully
    if os.path.exists(dest_path):
        print("File downloaded successfully!")
    else:
        raise ValueError("Error occurred while downloading the file.")

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, dest_path):
    CHUNK_SIZE = 32768

    with open(dest_path, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

# Download the video from Google Drive
file_id = '1RLmqoMl0WTtng_vhvWt-ATHrGqzjtgr6'
dest_path = './Vancouver_Traffic_Intersection.MP4'

try:
    download_file_from_google_drive(file_id, dest_path)
except ValueError as e:
    print(str(e))


### Run inference on the source

In [4]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov9c.pt')

#results = model('a.jpg', save=True)
results = model('Vancouver_Traffic_Intersection.MP4', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/968) /Users/abidikshit/GitProjects/Yolov9-VancouverTrafficCongestion/Vancouver_Traffic_Intersection.MP4: 384x640 4 persons, 14 cars, 1 truck, 3 traffic lights, 1 horse, 598.1ms
video 1/1 (frame 2/968) /Users/abidikshit/GitProjects/Yolov9-VancouverTrafficCongestion/Vancouver_Traffic_Intersection.MP4: 384x640 4 persons, 13 cars, 1 truck, 3 traffic lights, 1 horse, 667.3ms
video 1/1 (frame 3/968) /Users/abidikshit/GitProjects/Yolov9-Van

### Import necessary modules for object detection and Load yolov9c.pt model

The YOLOv9 series introduces four models:

1. YOLOv9-s (small)
2. YOLOv9-m (medium)
3. YOLOv9-c (compact)
4. YOLOv9-e (extended)

### Now let's get the traffic count for respective incoming vehicle w.r.t the intersection directions

In [2]:
# Import necessary modules for object detection
import cv2
from tracker import Tracker
import pandas as pd
import math
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# Video file path
video_path = 'Vancouver_Traffic_Intersection.MP4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Coordinates for the red lines (nb: north bound; sb: south boung; wb: west bound)
nb_line_start = (880, 390)
nb_line_end = (1000, 380)

sb_line_start = (740, 660)
sb_line_end = (1010, 620)

wb_line_start = (630, 430)
wb_line_end = (630, 620)

# Define intervals for X and Y axes
x_interval = 50
y_interval = 50

# Initialize variables for counting vehicles
nb_vehicles_counted = set()
sb_vehicles_counted = set()
wb_vehicles_counted = set()

# Initialize the tracker
tracker = Tracker()

while True:    
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects in the frame
    results = model.predict(frame)
    objects = results[0].boxes.data.detach().cpu().numpy()

    # Prepare the list of bounding boxes for vehicles only
    bounding_boxes = []
    for obj in objects:
        x1, y1, x2, y2, _, class_id = obj
        class_id = int(class_id)  # Convert class ID to integer
        if class_id < len(class_list) and class_list[class_id] in ["car", "bus", "truck", "motorcycle"]:
            bounding_boxes.append([x1, y1, x2, y2])

    # Update the tracker with the detected bounding boxes
    tracked_objects = tracker.update(bounding_boxes)

    # Process each tracked object
    for tracked_obj in tracked_objects:
        x1, y1, x2, y2, obj_id = tracked_obj
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
    
        # Check if the object is within the range of the red lines
        # if nb_line_start[0] < cx < nb_line_end[0] and abs(cy - nb_line_start[1]) < 10:
        #     if obj_id not in nb_vehicles_counted:
        #         nb_vehicles_counted.add(obj_id)
        #         cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
    
        # elif sb_line_start[0] < cx < sb_line_end[0] and abs(cy - sb_line_end[1]) < 10:
        #     if obj_id not in sb_vehicles_counted:
        #         sb_vehicles_counted.add(obj_id)
        #         cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
    
        # elif wb_line_start[1] < cy < wb_line_end[1] and abs(cx - wb_line_start[0]) < 10:
        #     if obj_id not in wb_vehicles_counted:
        #         wb_vehicles_counted.add(obj_id)
        #         cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
        # Check if the object is within the range of the red lines
        
        if nb_line_start[0] < cx < nb_line_end[0] and abs(cy - nb_line_start[1]) < 10:
            if obj_id not in nb_vehicles_counted:
                nb_vehicles_counted.add(obj_id)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
            else:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle
        elif sb_line_start[0] < cx < sb_line_end[0] and abs(cy - sb_line_end[1]) < 10:
            if obj_id not in sb_vehicles_counted:
                sb_vehicles_counted.add(obj_id)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
            else:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle
        elif wb_line_start[1] < cy < wb_line_end[1] and abs(cx - wb_line_start[0]) < 10:
            if obj_id not in wb_vehicles_counted:
                wb_vehicles_counted.add(obj_id)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
            else:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle

        else:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle

                
    # Draw the red lines and labels
    red_color = (0, 0, 255)  # (B, G, R)   
    cv2.line(frame, (nb_line_start[0], nb_line_start[1] + 3), nb_line_end, red_color, 3)
    cv2.putText(frame, 'NB Incoming', nb_line_start, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA) 

    cv2.line(frame, (sb_line_start[0] - 3, sb_line_start[1]), sb_line_end, red_color, 3)
    cv2.putText(frame, 'SB Incoming', (sb_line_start[0] - 130, sb_line_start[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA) 

    cv2.line(frame, (wb_line_start[0], wb_line_start[1] + 3), wb_line_end, red_color, 3)
    cv2.putText(frame, 'WB Incoming', wb_line_start, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA) 

    # Display vehicle counts for each direction
    cv2.putText(frame, f'NB Vehicles: {len(nb_vehicles_counted)}', (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1, cv2.LINE_AA)
    cv2.putText(frame, f'SB Vehicles: {len(sb_vehicles_counted)}', (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1, cv2.LINE_AA)
    cv2.putText(frame, f'WB Vehicles: {len(wb_vehicles_counted)}', (60, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1, cv2.LINE_AA)
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 4 persons, 14 cars, 1 truck, 3 traffic lights, 1 horse, 508.8ms
Speed: 4.8ms preprocess, 508.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 13 cars, 1 truck, 3 traffic lights, 1 horse, 589.4ms
Speed: 3.3ms preprocess, 589.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 15 cars, 1 bus, 2 trucks, 4 traffic lights, 515.7ms
Speed: 4.1ms preprocess, 515.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 14 cars, 1 truck, 3 traffic lights, 483.0ms
Speed: 3.0ms preprocess, 483.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 16 cars, 1 truck, 4 traffic lights, 508.4ms
Speed: 3.3ms preprocess, 508.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 16 cars, 1 truck, 4 traffic lights, 501.8ms
Speed: 4.0ms preprocess, 501.8ms inference, 0.9ms postprocess per image at shape (

In [7]:
# # Import necessary modules for object detection

# import cv2
# import pandas as pd
# import math
# from ultralytics import YOLO

# # Build a YOLOv9c model from pretrained weight
# model = YOLO('yolov9c.pt')


# class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


# import cv2
# from tracker import Tracker

# # Video file path
# video_path = 'Traffic.MP4'

# # Open the video file
# cap = cv2.VideoCapture(video_path)

# # Coordinates for the red lines (nb: north bound; sb: south boung; wb: west bound)
# nb_line_start = (880, 390)
# nb_line_end = (1000, 380)

# sb_line_start = (740, 660)
# sb_line_end = (1010, 620)

# wb_line_start = (630, 430)
# wb_line_end = (630, 620)

# # Define intervals for X and Y axes
# x_interval = 50
# y_interval = 50

# # Initialize variables for counting vehicles
# nb_vehicles_counted = set()
# sb_vehicles_counted = set()
# wb_vehicles_counted = set()

# # Initialize the tracker
# tracker = Tracker()

# while True:    
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Detect objects in the frame
#     results = model.predict(frame)
#     objects = results[0].boxes.data.detach().cpu().numpy()

#     # Prepare the list of bounding boxes for vehicles only
#     bounding_boxes = []
#     for obj in objects:
#         x1, y1, x2, y2, _, class_id = obj
#         class_id = int(class_id)  # Convert class ID to integer
#         if class_id < len(class_list) and class_list[class_id] in ["car", "bus", "truck", "motorcycle"]:
#             bounding_boxes.append([x1, y1, x2, y2])

#     # Update the tracker with the detected bounding boxes
#     tracked_objects = tracker.update(bounding_boxes)

#     for tracked_obj in tracked_objects:
#         x1, y1, x2, y2, obj_id = tracked_obj
#         cx = (x1 + x2) // 2
#         cy = (y1 + y2) // 2
    
#         # Check if the object is within the range of the red lines
#         if nb_line_start[0] < cx < nb_line_end[0] and abs(cy - nb_line_start[1]) < 10:
#             if obj_id not in nb_vehicles_counted:
#                 nb_vehicles_counted.add(obj_id)
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
#             else:
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle
#         elif sb_line_start[0] < cx < sb_line_end[0] and abs(cy - sb_line_end[1]) < 10:
#             if obj_id not in sb_vehicles_counted:
#                 sb_vehicles_counted.add(obj_id)
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
#             else:
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle
#         elif wb_line_start[1] < cy < wb_line_end[1] and abs(cx - wb_line_start[0]) < 10:
#             if obj_id not in wb_vehicles_counted:
#                 wb_vehicles_counted.add(obj_id)
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Draw red rectangle around the vehicle
#             else:
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle

#         else:
#             cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw green rectangle around the vehicle

#     # Draw the red lines and labels
#     red_color = (0, 0, 255)  # (B, G, R)   
#     cv2.line(frame, (nb_line_start[0], nb_line_start[1] + 3), nb_line_end, red_color, 3)
#     cv2.putText(frame, 'NB Incoming', nb_line_start, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA) 

#     cv2.line(frame, (sb_line_start[0] - 3, sb_line_start[1]), sb_line_end, red_color, 3)
#     cv2.putText(frame, 'SB Incoming', (sb_line_start[0] - 130, sb_line_start[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA) 

#     cv2.line(frame, (wb_line_start[0], wb_line_start[1] + 3), wb_line_end, red_color, 3)
#     cv2.putText(frame, 'WB Incoming', wb_line_start, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA) 

#     # Display vehicle counts for each direction
#     cv2.putText(frame, f'NB Vehicles: {len(nb_vehicles_counted)}', (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1, cv2.LINE_AA)
#     cv2.putText(frame, f'SB Vehicles: {len(sb_vehicles_counted)}', (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1, cv2.LINE_AA)
#     cv2.putText(frame, f'WB Vehicles: {len(wb_vehicles_counted)}', (60, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1, cv2.LINE_AA)
    
#     cv2.imshow("frames", frame)
#     if cv2.waitKey(1) & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()
